# 05: Delaunay Triangulation

*Authors: Felix Espey*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction
2. Edge Flips
3. Delaunay Triangulation
   1. Randomised Incremental Construction
4. Voronoi diagrams
5. References

# 1. Introduction

This chapter focuses mainly on the **Delaunay Triangulation**, but also contains a small section on the connected topic of **Voronoi diagrams**.

We start by laying out some concepts from the lecture that are important to the Delaunay Triangulation and then explain the algorithm to construct the Delaunay Triangulation from the lecture.

# 2. Edge Flips

As the name suggests, edge flips refers to the process of flipping the edge between two triangles to the other possible configuration.

The image below shows how flipping the edge *pq* between the triangles *pqr* and *pqs* increases the smallest angle within the triangle. An edge that can be flipped to increase the smallest angle of a triangulation is called illegal, while edges for which this is not the case are called legal.

<img style='float: left;' src='./images/06-image00.PNG'>

The most intuitive way to see if an edge is illegal is by calculating the inner angles of the triangles before and after the flip and checking if the smallest angle increases. 

A simple way to test wheter an edge *pq* is illegal is to calculate the circumcircle C<sub>p,q,r</sub> of the all Triangles *pqr* is empty. A circle is defined by a boundary and an interior, and a circle is empty if no points are within its interior. The circle C<sub>p,q,r</sub> is defined as the circle whose boundary goes through the points p, q, and r. Given two triangles, *pqr* and *pqs*, the edge *pq* between them is illegal if the point *s* is in the circumcircle of the triangle *pqr*.

We will now use this to highlight all illegal edges in a given triangulation.

## 2.1 Visualization

The visualization below is initialized with three far-away points that form a triangle outside the screen. Adding a point will split the face it is in into three faces by adding a new vertex at the click location and edges to the three vertices of that face. This ensures the triangulation stays intact when points are added. The algorithm will iterate over all edges once and highlight the ones that are illegal in blue. Turning on animations will also show the circumcircle calculated to check the legality of an edge.

It is important to note that not all edges highlighted this way would actually be flipped if we chose to flip all illegal edges, since some edges are only illegal until a neighboring edge is flipped. Additionally, flipping all highlighted edges would not make the triangulation legal afterward, as new illegal edges can be created when other edges are flipped.

In [1]:
from modules.visualisation import VisualisationTool, TriangleInstance, TriangleMode, IllegalEdgeMode
from modules.data_structures import Triangulation
from modules.data_structures.animation_objects import EdgeAnimator
from modules.geometry import Point, PointList

def highlight_illegal_edges(t : Triangulation) -> EdgeAnimator:
    animator = EdgeAnimator()
    for e in t.edges:
        #skip all outer edges as they have no second triangle and therefore can not be illegal
        if e.incident_face is t.outer_face or e.twin.incident_face is t.outer_face:
            continue
        #skip edge if its twin was already checked 
        if animator.edge_or_twin_checked(e):
            continue
        animator.highlight_triangle(e)
        is_legal = t.is_legal(e)
        if not is_legal:
            animator.add_illegal_edge(e)
        animator.un_highlight_triangle(is_legal)
        animator.add_checked_edge(e)
    return animator

triangle_instance = TriangleInstance(Point(0, 399), Point(399, 399), Point(199, 0))
triangle_instance._default_number_of_random_points = 20
vis1 = VisualisationTool(400, 400, triangle_instance)


illegal_edge_mode = IllegalEdgeMode()
vis1.register_algorithm("highlight illegal edges", highlight_illegal_edges, illegal_edge_mode)
vis1.display()
vis1.add_points([Point(0, 399), Point(399, 399), Point(199, 0)])

# 3. Delaunay Triangulation

The Delaunay Triangulation is a special case of triangulation in which all edges are legal. Based on the previous section, we already established that an edge is legal if the corresponding circumcircle is empty. This leads directly to the **Empty Circle Property** from the lecture, which states that in a Delaunay Triangulation, the circumcircle C<sub>p,q,r</sub> of all triangles contains no other points, and a triangulation in which the circumcircles C<sub>p,q,r</sub> of all triangles p,q,r are empty is a Delaunay Triangulation.

## 3.1 Randomized Incremental Construction

The lecture presented a Randomized Incremental Construction Algorithm to construct the Delaunay Triangulation given a set of points. The points are added one by one in a random order by splitting the containing triangle into three smaller triangles. After each insertion, all new edges are checked for legality and flipped if necessary. If an edge is flipped then adjacent edges are checked recursivly until the no further flips occour. 

As with the visualization before, the algorithm starts with three points far away, with the edges to those points being drawn transparent.

In [2]:
from modules.data_structures.animation_objects import IncrementalConstructionAnimator as RICA
from modules.visualisation import PointSetInstance
import random

triangle_mode = TriangleMode(Point(-4000, 4000),Point(4000, 4000),Point(200,-4000))
triangle_mode._draw_outer_points = False

def randomized_incremental_construction(points : set[Point]) -> RICA:
    animator = RICA(Point(-4000, 4000),Point(4000, 4000),Point(200,-4000))
    points = list(points)
    random.shuffle(points)
    for p in points:
        v = animator.insert_point(p)
        if v is None:
            continue
        edges = v.outgoing_edges()
        for e in edges:
            animator.legalize_edge(e.next, v)
    return animator

point_instance = PointSetInstance()
point_instance._default_number_of_random_points = 20

vis2 = VisualisationTool(400, 400, point_instance)
vis2.register_algorithm("Randomzied Incremental Construction", randomized_incremental_construction, triangle_mode)
vis2.display()

# 4. Voronoi Diagramms

In the lecture, the Delaunay Triangulation was introduced as the dual graph of the Voronoi Diagram of a set of points. Since Voronoi diagrams were only a small part of the lecture and primarily served as an introduction, we will only take a short look at them here.

## 4.1 Definition

A Voronoi diagram is a subdivision of the plane into cells through a set of points *P*. The cell of a point p is defined as the set of all points for which no other points from *P* are closer than p. This means points in the plane for which two points in *P* are equally close are not part of any cell but instead define the boundary edges between cells. Points with three or more points from *P* equally far away become Voronoi vertices.

## 4.2 Relation to the Delaunay Triangulation

As stated before, the Delaunay Triangulation is the dual graph of the Voronoi Diagram. This means that, given a Voronoi diagram, one can get the Delaunay triangulation by connecting all points in *P* whose cells are adjacent. One can also calculate the Voronoi diagram given the Delaunay triangulation.

In [3]:
from modules.data_structures.animation_objects import VoronoiAnimator


def voronoi_construction(points : set[Point]) -> VoronoiAnimator:
    delaunay_animator = randomized_incremental_construction(points)
    animator = VoronoiAnimator(delaunay_animator)
    for face in delaunay_animator.faces:
        #skip all connections to the dummy vertices
        if animator.is_inner_face(face):
            p = Triangulation.center_of_circumcircle(face.outer_points()[0], face.outer_points()[1], face.outer_points()[2])
            animator.add_point(p, face)
    return animator

vis2.register_algorithm("Voronoi Diagram", voronoi_construction, triangle_mode)
vis2.display()

# 5.References

\[1\] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd edition. 2008.